https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
from scipy.io import wavfile
from scipy.ndimage.interpolation import shift
from scipy.signal import hamming

data = pd.read_csv("../../Dataset/vox1_meta.csv", delimiter = '\t', index_col = "VoxCeleb1 ID")

data.head()

,VGGFace1 ID,Gender,Nationality,Set
VoxCeleb1 ID,,,,
id10001,A.J._Buckley,m,Ireland,dev
id10002,A.R._Rahman,m,India,dev
id10003,Aamir_Khan,m,India,dev
id10004,Aaron_Tveit,m,USA,dev
id10005,Aaron_Yoo,m,USA,dev


In [86]:
dev_loc = "../../Dataset/vox1_dev_wav/vox1_dev_wav/wav"

In [87]:
sample_wav_file = wavfile.read(os.path.join(dev_loc, 'id10001/1zcIwhmdeo4/00002' + '.wav'))

In [88]:
def apply_preemphasis(wav_file, alpha):
    amplitudes = np.array(wav_file[1])
    shifted = shift(amplitudes, 1, cval=0)
    return (wav_file[0], amplitudes - alpha * shifted)

In [82]:
def apply_windowing(wav_file, window_size=3, keep_remainder=False):
    amplitudes = np.array(wav_file[1])
    sample_rate = wav_file[0]
    sub_arrays = np.split(amplitudes, range(0,len(amplitudes), sample_rate * window_size)[1:])
    if keep_remainder:
        return sub_arrays
    elif len(sub_arrays[-1]) == sample_rate * window_size:
        return sub_arrays
    else:
        return sub_arrays[:-1]

In [89]:
def apply_hamming(signal_vector, size):
    return signal_vector * hamming(size)

for subs in apply_windowing(sample_wav_file):
    print(apply_hamming(subs, sample_wav_file[0] * 3))

[455.68       319.92001576 266.80005257 ... 584.48011518 647.20003188
 593.2       ]
[546.96       453.68002235 284.32005603 ... 210.72004152 192.8000095
  18.4       ]


In [ ]:
low_freq_mel = 0
high_freq_mel = (2595 * np.log10(1+(sample_rate/2)/700))
mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)
hz_points = (700 * (10**(mel_points/2595) - 1))
bin = np.floor((NFFT + 1) * hz_points / sample_rate)

fbank = np.zeros((nfilt, int(np.floor(NFFT /2 + 1))))
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])